<font size="5"><b>Lecture 08: Machine Learning 2/3 (exercises)</b></font>

# EX: Semantic classification of satellite image (part 1/2)

<div class="alert alert-warning">
<b>This exercice has been developped by <u>Andreas Ley</u> and <u>Ronny Hänsch</u> (TU-Berlin) in the framework of the GEO.X 2017 Autumn school.<br>
<br>
The exercice is designed to implement the classification of a satellite image (Sentinel-2) into semantic labels defining land use: forest, fields, urban, water.<br>
<br>
The exercice is split into 2 parts: this first part (lecture 08) will implement a PCA analisis on the image crops in order to reduce dimensionality. The second part (lecture 09) will use the outputs of the PCA and implement a SVM to classify the pixels into different land use classes.
    </b>
</div>

<div class="alert alert-warning">
<b><u>Description of the dataset</u></b>:<br>
<br>
The data stored in the file "images/s2_training_data.npz" contains image crops of a Sentinel-2 multispectral image, and some semantic labels of land use.<br>
    <br>
    The file can be loaded using <a href="https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load" target="_blank">numpy.load()</a>, and contains 2 parts:<br>
    <ul>
        <li><u>data</u>: contains 20,000 patches (crops), each of 15x15 pixels and 4 channels (R + G + B + shortwave-infrared). The data is organized as one numpy array of shape (20000, 4, 15, 15), where the first dimension are the instances, the next dimension are the channels, and the latter two are width and height.<br> The first 5000 crops contain forest in the central pixel, the next 5000 contain fields/lower vegetation, the next 5000 are urban areas, and the last 5000 are water.</li>
        <li><u>labels</u>: contains semantic labels of land use (forest, fields, urban, water), which will be later used for training the classifier. These can be ignored for now.</li>
    </ul>   
</div>

## load data

<div class="alert alert-success">
    Load the data using numpy.load(). Store the "data" in a variable named "train_X", anf the "labels" in a variable named "train_Y". Explore the shape of these arrays.
</div>

## inspect image crops

<div class="alert alert-success">
    Use the function below to look at the (RGB-part) of individual crops.<br>Take a look at the first crop of each class (remember: crops 1-5k contain forest, crops 5-10k contain fields, crops 10-15k are urban areas, and crops 15-20k are water).
</div>

In [ ]:
def show_raw_image(img):
    img2 = np.log(img[[2,1,0],:,:])

    img2[0,:,:] = img2[0,:,:].copy() * 1.05303 + -6.32792
    img2[1,:,:] = img2[1,:,:].copy() * 1.74001 + -10.8407
    img2[2,:,:] = img2[2,:,:].copy() * 1.20697 + -6.73016

    img2 = np.clip(img2 / 6 + 0.5, 0.0, 1.0)

    plt.imshow(np.transpose(img2, (1, 2, 0)))
    plt.show()

## compress image crops

<div class="alert alert-success">
    The data train_X contains the raw values which span a large range. To compress them into a more “gaussian” shape, compute the logarithm of them.
</div>

## build function to compute principal components of image crops
The following tasks will implement a function "compute_mean_PCs(X)" which takes image crops as inputs, and returns the mean image, Eigen images (principal components reshaped into images), and Eigen values.

### compute mean crop, and mean-free crops

<div class="alert alert-success">
    Compute the mean image crop, and name the resulting array "mean".<br>
    Subtract this "mean" from all crops, and name the resulting array "mean_free".<br>
    <br>
    Hint: The mean image should be an array of shape (4, 15, 15), and the mean free crops should be an array of shape (20000, 4, 15, 15)).
</div>

### vectorize the mean-free crops

<div class="alert alert-success">
    Flatten the crops into vectors so that you get a numpy array of shape (20000, 4 * 15 * 15).<br>
    <br>
    Hint: use "np.reshape".
</div>

### compute the covariance matrix

<div class="alert alert-success">
    Compute the covariance matrix. This should be a matrix of size 900x900 (900 = 4 * 15 * 15).<br>
    <br>
    Start by defining a matrix of zeros with shape (900,900). Loop over all 20000 crops, and for each crop compute the outer product <a href="https://numpy.org/doc/stable/reference/generated/numpy.outer.html?highlight=outer#numpy.outer" target="_blank">np.outer(u, v)</a> of the mean-free vector with itself (i.e., u = v = vectorized crop). Accumulate (sum) the outer products together and finally divide by the total number of used crops. (To speed up development, consider using only every 10th crop).<br>
    <br>
    NB: the <a href="https://en.wikipedia.org/wiki/Outer_product" target="_blank">outer product</a> $u \otimes v$ is equivalent to a matrix multiplication $u \cdot v^T$
</div>

### compute Eigen values & Eigen vectors of the covariance matrix

<div class="alert alert-success">
    Compute the Eigen values "eig_val" and Eigen vectors "eig_vec" of the covariance matrix using <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html?highlight=eig#numpy.linalg.eig" target="_blank">np.linalg.eig()</a>.<br>
    <br>
    Sort them by importance (decreasing Eigen values) using the following code:<br>
</div>

```python
# Sort by importance
idx = np.argsort(eig_val)[::-1]
eig_vec = eig_vec[:,idx]
eig_val = eig_val[idx]
```

### reshape Eigen vectors into Eigen images

<div class="alert alert-success">
    Reshape the 900 eigen vectors back into the images, i.e. with shape (900, 4, 15, 15), and name this variable "principal_components".<br>
    <br>
    Note that the matrix eig_vec contains the Eigen vectors in its rows (not columns as you might expect), so you'll need to transpose the matrix before reshaping it.
</div>

### merge into function "compute_mean_PCs"

<div class="alert alert-success">
    Merge steps 1.4.1 to 1.4.4 into a single function "compute_mean_PCs".<br>
    <br>
    The function will take the image crops as inputs (i.e., variable "train_X"), and will return the mean image (i.e., variable "mean"), the Eigen images (principal components reshaped into images, i.e., variable "principal_components"), and Eigen values (i.e., variable "eig_val").
</div>

## run "compute_mean_PCs" and plot Eigen values

<div class="alert alert-success">
    Run "compute_mean_PCs" on the satellite data, and plot the returned Eigen values.<br>
    <br>
    Look at the first 64 principal components (or the RGB part thereof) using the following function:
</div>

```python
def show_first_principal_components(pcs):

    f, axarr = plt.subplots(8,8)
    for i in range(0,8):
        for j in range(0,8):
            img2 = pcs[i*8+j, [2,1,0], :, :]
            img2 = np.clip(img2 * 10 + 0.5, 0.0, 1.0)
            axarr[i,j].imshow(np.transpose(img2, (1, 2, 0)))

    plt.show()
```

## compute features

<div class="alert alert-success">
    Write a function "compute_features(X, mean, principal_components, count)" that takes crops "X", subtracts the mean, and projects them onto the first "count" principal components.<br>
    <br>
    The returned array should have a shape (X.shape[0], count), containing the coefficients (which we will use as features in next week's exercise).<br>
    <br>
    Hint: the projection is done using the dot product.
</div>

## reconstruct image

<div class="alert alert-success">
    Write a function "reconstruct_image(feature, mean, principal_components)" that restores a crop given a feature/coefficient vector.<br>
    <br>
    Use the following code to compare, side by side, original image crops and reconstructions:
</div>

In [ ]:
# Code to compare, side by side, original image crops and reconstructions: 
for i in range(0,4):    
    img = np.concatenate((train_X[5000*i+0,:,:,:], reconstruct_image(train_features[5000*i+0,:], mean, principal_components)), 2);
    img = np.concatenate((img,np.concatenate((train_X[5000*i+1,:,:,:], reconstruct_image(train_features[5000*i+1,:], mean, principal_components)), 2)), 1);
    img = np.concatenate((img,np.concatenate((train_X[5000*i+2,:,:,:], reconstruct_image(train_features[5000*i+2,:], mean, principal_components)), 2)), 1);
    img = np.concatenate((img,np.concatenate((train_X[5000*i+3,:,:,:], reconstruct_image(train_features[5000*i+3,:], mean, principal_components)), 2)), 1);
    img = np.concatenate((img,np.concatenate((train_X[5000*i+4,:,:,:], reconstruct_image(train_features[5000*i+4,:], mean, principal_components)), 2)), 1);
    show_raw_image(np.exp(img))